## Imports

In [153]:
import random
import math
import time
from base64 import b64encode, b64decode


## Utils

El código fue sacado del siguiente repositorio: https://github.com/marceloarenassaavedra/IIC2283-2-21/blob/master/codigos%20de%20clases/alg_teoria_numeros.py

In [154]:
def _exp(a: int, b: int) -> int:
    """
    Argumentos :
        a: int
        b: int - b >= 0
    Retorna :
        int - a**b
    """
    if b == 0:
        return 1
    else:
        res = 1
        pot = a
        while b > 0:
            if b % 2 == 1:
                res = pot * res
            b = b // 2
            pot = pot * pot
        return res



def _exp_mod(a: int, b: int, n: int) -> int:
    """
    Argumentos :
        a: int
        b: int
        n: int - n > 0
    Retorna :
        int - a**b en modulo n
    """
    if b == 0:
        return 1
    elif b > 0:
        res = 1
        pot = a
        while b > 0:
            if b % 2 == 1:
                res = (pot * res) % n
            b = b // 2
            pot = (pot * pot) % n
        return res
    else:
        return _exp_mod(_inverso(a,n),-b,n)

    

def _mcd(a: int, b: int) -> int:
    """
    Argumentos :
        a: int
        b: int - a > 0 o b > 0
    Retorna :
        maximo comun divisor entre a y b,
    """
    while b > 0:
        temp = b
        b = a % b
        a = temp
    return a



def _alg_ext_euclides(a: int, b: int) -> (int, int, int):
    """
    Argumentos :
        a: int
        b: int - a >= b >= 0 y a > 0
    Retorna :
        (int , int , int) - maximo comun divisor MCD(a, b) entre a y b,
        y numeros enteros s y t tales que MCD(a, b) = s*a + t*b
    """
    r_0 = a
    s_0 = 1
    t_0 = 0
    r_1 = b
    s_1 = 0
    t_1 = 1
    while r_1 > 0:
        r_2 = r_0 % r_1
        s_2 = s_0 - (r_0 // r_1) * s_1
        t_2 = t_0 - (r_0 // r_1) * t_1
        r_0 = r_1
        s_0 = s_1
        t_0 = t_1
        r_1 = r_2
        s_1 = s_2
        t_1 = t_2
    return r_0, s_0, t_0



def _inverso(a: int, n: int) -> int:
    """
    Argumentos :
        a: int - a >= 1
        n: int - n >= 2, a y n son primos relativos
    Retorna :
        int - inverso de a en modulo n
    """
    (r, s, t) = _alg_ext_euclides(a, n)
    return s % n



def _es_potencia(n: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
    Retorna :
        bool - True si existen numeros naturales a y b tales que n = (a**b),
        donde a >= 2 y b >= 2. En caso contrario retorna False.       
    """
    if n <= 3:
        return False
    else:
        k = 2
        lim = 4
        while lim <= n:
            if _tiene_raiz_entera(n, k):
                return True
            k = k + 1
            lim = lim * 2
        return False


    
def _tiene_raiz_entera(n: int, k: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
        k: int - k >= 2
    Retorna :
        bool - True si existe numero natural a tal que n = (a**k),
        donde a >= 2. En caso contrario retorna False.       
    """
    if n <= 3:
        return False
    else:
        a = 1
        while _exp(a,k) < n:
            a = 2*a
        return _tiene_raiz_entera_intervalo(n, k, a//2, a)


    
def _tiene_raiz_entera_intervalo(n: int, k: int, i: int, j: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
        k: int - k >= 2
        i: int - i >= 0
        j: int - j >= 0
    Retorna :
        bool - True si existe numero natural a tal que n = (a**k),
        donde i <= a <= j. En caso contrario retorna False.       
    """
    while i <= j:
        if i==j:
            return n == _exp(i,k)
        else:
            p = (i + j)//2 
            val = _exp(p,k)
            if n == val:
                return True
            elif val < n:
                i = p+1
            else:
                j = p-1
    return False



def _test_primalidad(n: int, k: int) -> bool:
    """
    Argumentos :
        n: int - n >= 1
        k: int - k >= 1
    Retorna :
        bool - True si n es un numero primo, y False en caso contrario.
        La probabilidad de error del test es menor o igual a 2**(-k),
        y esta basado en el test de primalidad de Solovay–Strassen
    """
    if n == 1:
        return False
    elif n == 2:
        return True
    elif n%2 == 0:
        return False
    elif _es_potencia(n):
        return False
    else:
        neg = 0
        for i in range(1,k+1):
            a = random.randint(2,n-1)
            if _mcd(a,n) > 1:
                return False
            else:
                b = _exp_mod(a,(n-1)//2,n)
                if b == n - 1:
                    neg = neg + 1
                elif b != 1:
                    return False
        if neg > 0:
            return True
        else:
            return False

In [155]:
first_100_prime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]

## Code

In [156]:
class RSAReceiver:
    def __init__(self, bit_len: int) -> None:
        """
        Arguments :
            bit_len : A lower bound for the number of bits of N,
            the second argument of the public and secret key .
        """
        self.bit_len = bit_len
        self.private_key = None

    def get_public_key(self) -> bytearray:
        """
        Returns :
            public_key : Public key expressed as a Python ’bytearray ’ using the
            PEM format . This means the public key is divided in:
            (1) The number of bytes of e (4 bytes )
            (2) the number e (as many bytes as indicated in (1))
            (3) The number of bytes of N (4 bytes )
            (4) the number N (as many bytes as indicated in (3))
        """
        init_time = time.time()
        # Generate primes to generate the public key
        primes = []
        min_num = pow(2, (self.bit_len // 2))
        max_num = pow(2, self.bit_len // 2 + 7)
        while len(primes) < 2:
            # If we already have two primes, we can stop
            if len(primes) == 2:
                break
            # Generate a random number
            num = random.randint(min_num, max_num)
            is_prime = True
            for i in first_100_prime:
                if num % i == 0:
                    is_prime = False
                    break
            if not is_prime:
                continue
            # If it is prime, add it to the list
            if _test_primalidad(num, 100):
                primes.append(num)
        # Generate the public key
        N = primes[0] * primes[1]
        phi_N = (primes[0] - 1) * (primes[1] - 1)
        d = 0
        # Generate d
        while d == 0:
            num = random.randint(0, N)
            if _mcd(num, phi_N) == 1:
                d = num
        # Generate e
        e = _inverso(d, phi_N)
        self.private_key = (d, N)
        # Generate public key
        e_len = int((e.bit_length() + 7) // 8).to_bytes(4, "big")
        N_len = int((N.bit_length() + 7) // 8).to_bytes(4, "big")
        return (
            bytearray(e_len)
            + bytearray(e.to_bytes(int.from_bytes(e_len, "big"), "big"))
            + bytearray(N_len)
            + bytearray(N.to_bytes(int.from_bytes(N_len, "big"), "big"))
        )

    def decrypt(self, ciphertext: bytearray) -> str:
        """
        Arguments :
            ciphertext : The ciphertext to decrypt
        Returns :
            message : The original message
        """
        assert self.private_key is not None, "No private key. You can generate one with get_public_key() method"
        # Get the private key
        d, N = self.private_key
        # Get N bits number
        N_bits_num = N.bit_length()
        # Get block size
        if N_bits_num % 8 == 0:
            n = N_bits_num // 8
        else:            
            n = math.floor(N_bits_num / 8) + 1
        # Encoded message
        decoded_message = bytearray()
        # Decrypt the message
        for i in range(0, len(ciphertext), n):
            # Get the block
            block = ciphertext[i : i + n]
            # Get the number
            num = int.from_bytes(block, "big")
            # Get the message
            message = _exp_mod(num, d, N)
            # Get message bytes number
            message_len = (message.bit_length() + 7) // 8
            # Convert the message to bytes
            message = message.to_bytes(message_len, "big")
            # Append the message to the decoded message
            decoded_message += message
        return decoded_message.decode("utf-8")

        


In [157]:
class RSASender:
    def __init__(self, public_key: bytearray) -> None:
        """
        Arguments :
            public_key : The public key that will be used to encrypt messages
        """
        self.public_key = public_key

    def encrypt(self, message: str) -> bytearray:
        """
        Arguments :
            message : The plaintext message to encrypt
        Returns :
            ciphertext : The encrypted message
        """
        ciphertext = bytearray()
        # Get bytearray from message
        message_bytes = bytearray(message, "utf-8")
        # Get e length
        e_len = int.from_bytes(self.public_key[:4], "big")
        # Get e
        e = int.from_bytes(self.public_key[4:4 + e_len], "big")
        # Get N length
        N_len = int.from_bytes(self.public_key[4 + e_len: 8 + e_len], "big")
        # Get N 
        N = int.from_bytes(self.public_key[8 + e_len:], "big")
        # Get N bits number
        N_bits_num = N.bit_length()
        # Get block size
        if N_bits_num % 8 == 0:
            n = N_bits_num // 8 - 1
        else:            
            n = math.floor(N_bits_num / 8)
        #Encrypt message
        for i in range(0, len(message_bytes), n):
            # Get block
            block = message_bytes[i:i+n]
            # Get block numeric value
            block_number = int.from_bytes(block, "big")
            # Encrypt block
            encrypted_block = _exp_mod(block_number, e, N)
            # Add encrypted block to ciphertext
            ciphertext += encrypted_block.to_bytes(n + 1, "big")
        return ciphertext
        



In [161]:
if __name__ == "__main__":
    receiver = RSAReceiver(2024)
    public_key = receiver.get_public_key()
    sender = RSASender(public_key)
    message = "Hola que tal como estas???? xao que haces hoy este mensaje esta encriptado con RSA"
    ciphertext = sender.encrypt(message)
    assert message == receiver.decrypt(ciphertext), "The message is not the same"

In [163]:
if __name__ == "__main__":
    public_key = b64decode('AAAAQQGHaihgiufnjzyLXufDjUCGuaHrsUL+hCF/pMFHPoh+ZVi/2bMFh6oelzElVklsJ9mglyQjJIKAb1JB9mvtaEkLAAAAQQHIuF+wIJw6uzq8uXpW/QmsNjtBJ8HCJJcu2h7sDX18nc2qWYDWTfMiXPmPRvhkkz4A0oXTAMDP9xsxUIjYQNsx')
    text = (
        'Being open source means anyone can independently review '
        'the code. If it was closed source, nobody could verify the '
        'security. I think it’s essential for a program of this '
        'nature to be open source.'
    )
    sender = RSASender(public_key)
    cipher = sender.encrypt(text)
    assert b64encode(cipher) == b'ALwPm7JXWbqGeIflV8PYgprs6mSgCH2Ydy0rgvFolzY0mczKItlPSHueL54uvDJXIz9pXoHZGAOPWVYYbcwRh3EBl8pi3MraUC2BBFUviMPFwNMwza/QMd5DNG9tH8doHlLRRt+15wLrsIE+m5T8fuM4HHixSNcEoOdN8T++q0PkzQDXL+UgbusiD3J+QPO59aqAB5HFcZ7P5U3fhFS8Qm1vLG8vlIulCby0jGLgjTtLUhFD/QhAof0y4F20gxedQDHwAOIrz6PEoBWnHmwLU0QNN0Rs542RvJ8BeEGhBDS5ZvD0/0Ix3ZqKT6HtP4ugfPD75/5LYGioJBwrg2DXbQucFj8='